# Cell Magics in IPython

IPython has a system of commands we call 'magics' that provide a mini command language that is orthogonal to the syntax of Python and is extensible by the user with new commands.  Magics are meant to be typed interactively, so they use command-line conventions, such as using whitespace for separating arguments, dashes for options and other conventions typical of a command-line environment.

Magics come in two kinds:

* Line magics: these are commands prepended by one `%` character and whose arguments only extend to the end of the current line.
* Cell magics: these use *two* percent characters as a marker (`%%`), and they receive as argument *both* the current line where they are declared and the whole body of the cell.  Note that cell magics can *only* be used as the first line in a cell, and as a general principle they can't be 'stacked' (i.e. you can only use one cell magic per cell).  A few of them, because of how they operate, can be stacked, but that is something you will discover on a case by case basis.

The `%lsmagic` magic is used to list all available magics, and it will show both line and cell magics currently defined:

In [1]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python  %%pyth

Since in the introductory section we already covered the most frequently used line magics, we will focus here on the cell magics, which offer a great amount of power.

Let's load matplotlib and numpy so we can use numerics/plotting at will later on.

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## <!--====--> Some simple cell magics

Timing the execution of code; the `timeit` magic exists both in line and cell form.

In the **line** form, the expression on the line is timed by executing it in a tight loop a number of times.

In [3]:
%timeit np.linalg.eigvals(np.random.rand(100,100))

6.39 ms ± 30.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In the **cell** form, the expression on the first line is executed only once, and acts as a setup for the body of the cell. The setup step is not timed. The body of the cell magic is executed in a tight loop, and the execution time returned.

In [5]:
%%timeit 
a = np.random.rand(100, 100)
np.linalg.eigvals(a)

6.49 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The `%%writefile` magic is a very useful tool that writes the cell contents as a named file:

In [17]:
%%writefile foo.py
print('Hello world')

Overwriting foo.py


In [18]:
%run foo

Hello world


## <!--====--> Magics for running code under other interpreters

IPython has a `%%script` cell magic, which lets you run a cell in
a subprocess of any interpreter on your system, such as: bash, ruby, perl, zsh, R, etc.

It can even be a script of your own, which expects input on stdin.

To use it, simply pass a path or shell command to the program you want to run on the `%%script` line,
and the rest of the cell will be run by that script, and stdout/err from the subprocess are captured and displayed.

In [19]:
%%script python2
import sys
print 'hello from Python %s' % sys.version

hello from Python 2.7.3 (default, Nov 19 2017, 01:35:09) 
[GCC 4.7.2]


In [20]:
%%script python3
import sys
print('hello from Python: %s' % sys.version)

hello from Python: 3.7.1 (default, Dec 14 2018, 19:28:38) 
[GCC 7.3.0]


IPython also creates aliases for a few common interpreters, such as bash, python, python2, python3, ruby, perl, etc.

These are all equivalent to `%%script <name>`

In [22]:
%%ruby
puts "Hello from Ruby #{RUBY_VERSION}"

Hello from Ruby 1.9.3


In [24]:
%%bash
echo "hello from $BASH"
ps

hello from /bin/bash
  PID TTY          TIME CMD
 1905 ?        00:00:00 dash
 1906 ?        00:00:00 invoke
 1907 ?        00:00:00 invoke_app
 2019 ?        00:00:00 pixelflip
 2021 ?        00:00:00 ratpoison
 2033 ?        00:00:00 python2
 2113 ?        00:00:00 start_jupyter
 2121 ?        00:00:00 xview
 2122 ?        00:00:18 ZMQbg/1
 2127 ?        00:00:01 ZMQbg/1
 3027 ?        00:00:01 ZMQbg/1
 3047 ?        00:00:01 ZMQbg/1
 3074 ?        00:00:00 ZMQbg/1
 3086 ?        00:00:01 ZMQbg/1
 3103 ?        00:00:00 ZMQbg/1
 3116 ?        00:00:00 ZMQbg/1
 3167 ?        00:03:15 ZMQbg/1
 3206 ?        00:00:00 bash
 3207 ?        00:00:00 ps


Keep in mind that when using the `%%script` magic, or its equivalents like `%%python` or `%%ruby`, the namespaces are not shared with the current interpreter.

## Capturing output

You can also capture stdout/err from these subprocesses into Python variables, instead of letting them go directly to stdout/err

In [25]:
%%bash
echo "hi, stdout"
echo "hello, stderr" >&2


hi, stdout


hello, stderr


In [26]:
%%bash --out output --err error
echo "hi, stdout"
echo "hello, stderr" >&2

In [27]:
print(error)
print(output)

hello, stderr

hi, stdout



## Background Scripts

These scripts can be run in the background, by adding the `--bg` flag.

When you do this, output is discarded unless you use the `--out/err`
flags to store output as above.

In [31]:
%%ruby --bg --out ruby_lines
for n in 1...10
    sleep 1
    puts "line #{n}"
    STDOUT.flush
end

When you do store output of a background thread, these are the stdout/err *pipes*,
rather than the text of the output.

In [32]:
ruby_lines

<_io.BufferedReader name=47>

In [33]:
print(ruby_lines.read().decode('utf8'))

line 1
line 2
line 3
line 4
line 5
line 6
line 7
line 8
line 9

